<div class="alert alert-success" role="alert">
<h1> Project Title: Development of Face Recognition System for Attendance using Python</h1>

<h2>Submitted by: Sachin Maruti Mundhe</h2>
</div>

# <font color= coral>Objective:
## Develop a system that uses face recognition to automate attendance logging in real-time.</font>

## Step 1: Set Up the Environment
Start by setting up the necessary libraries

In [2]:
!pip install opencv-python
!pip install face_recognition
!pip install pandas


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [41 lines of output]
  running bdist_wheel
  running build
  running build_ext
  
  
                     CMake is not installed on your system!
  
      Or it is possible some broken copy of cmake is installed on your system.
      It is unfortunately very common for python package managers to include
      broken copies of cmake.  So if the error above this refers to some file
      path to a cmake file inside a python or anaconda or miniconda path then you
      should delete that broken copy of cmake from your computer.
  
      Instead, please get an official copy of cmake from one of these known good
      sources of an official cmake:
          - cmake.org (this is how windows users should get cmake)
          - apt install cmake (for Ubuntu or Debian based systems)
          - yum install cmake (for Redhat or CenOS based systems)
  
      On a linux machine you c


  Obtaining dependency information for face_recognition from https://files.pythonhosted.org/packages/1e/95/f6c9330f54ab07bfa032bf3715c12455a381083125d8880c43cbe76bb3d0/face_recognition-1.3.0-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/100.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/100.1 MB 1.3 MB/s eta 0:01:19
     -------------------------------------- 0.0/100.1 MB 653.6 kB/s eta 0:02:34
     -------------------------------------- 0.1/100.1 MB 655.4 kB/s eta 0:02:33
     -------------------------------------- 0.2/100.1 MB 919.0 kB/s eta 0:01:49
     ---------------------------------------- 0.2/100.1 MB 1.0 MB/s eta 0:01:40
     ---------------------------------------- 0.4/100.1 MB 1.4 MB/s eta 0:01:10
     ---------------------------------------- 0.5/100.1 MB 1.5 MB/s eta 0:01:08
     ---------------------------------------- 0.6/100.1 MB 1.7 MB/s eta 0:01:00
     ---------------------------------------- 0.6/100.1 MB 1.7 MB

Defaulting to user installation because normal site-packages is not writeable


### A. Face Detection using OpenCV
This code captures a live video feed and detects faces in real-time.

In [1]:
import cv2

# Initialize webcam
video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    # Converting the frame to grayscale (for better performance in face detection)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Use OpenCV's pre-trained face detector
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

    # Display the resulting frame
    cv2.imshow('Face Detection', frame)

    # Exit the loop with 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
video_capture.release()
cv2.destroyAllWindows()


### B. Face Recognition using face_recognition library
This code recognizes faces and logs attendance.

## Step 2: Load Images for Recognition
First, load and encode images of people whose attendance needs to be recorded. 

In [ ]:
# Example of loading and encoding images
def load_and_encode_image(image_path):
    image = face_recognition.load_image_file(image_path)
    image_encoding = face_recognition.face_encodings(image)[0]
    return image_encoding

# Loading my reference images (known faces)
known_face_encodings = [
    load_and_encode_image("C:\My files\Study\Jobs\The_Research_School\Project\Face_Recognition_Attendance_Projects_main\Face-Recognition-Attendance-Projects-main\Training_images\Sachin.jpg"),  # Path to your image
    load_and_encode_image("C:\My files\Study\Jobs\The_Research_School\Project\Face_Recognition_Attendance_Projects_main\Face-Recognition-Attendance-Projects-main\Training_images\Modiji.png"),
]
known_face_names = ['Sachin', 'Modiji']  # Names of the individuals


## Step 3: Start Video Stream and Detect Faces
I have implemented the real-time face detection using OpenCV. By Using the webcam feed to capture video.

In [ ]:
# Opening the video stream from webcam
video_capture = cv2.VideoCapture(0)

# Initializing variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    # Capturing a single frame
    ret, frame = video_capture.read()
    
    # Resizing frame to improve speed
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    
    # Converting the image from BGR (OpenCV default) to RGB
    rgb_small_frame = small_frame[:, :, ::-1]
    
    # Processing every other frame for speed improvement
    if process_this_frame:
        # Finding face locations and face encodings in the current frame
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        
        face_names = []
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"
            
            # If a match is found in known_face_encodings
            if True in matches:
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]
            
            face_names.append(name)

    process_this_frame = not process_this_frame

    # Displaying the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scaling back up since the frame was resized
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Drawing a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Labeling the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Displaying the resulting image
    cv2.imshow('Video', frame)

    # Breaking from loop with 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Releasing the webcam
video_capture.release()
cv2.destroyAllWindows()


## Step 4: Marking Attendance in CSV File
When a face is recognized, I have loged the attendance with the current date and time.

In [ ]:
# Created a function to mark attendance
def mark_attendance(name):
    # Reading the existing attendance CSV (or create a new one)
    try:
        attendance_data = pd.read_csv('attendance_logs.csv')
    except FileNotFoundError:
        attendance_data = pd.DataFrame(columns=['Name', 'Date', 'Time'])

    now = datetime.now()
    date_str = now.strftime('%Y-%m-%d')
    time_str = now.strftime('%H:%M:%S')

    # Checking if the person is already marked for the day
    if not ((attendance_data['Name'] == name) & (attendance_data['Date'] == date_str)).any():
        attendance_data = attendance_data.append({'Name': name, 'Date': date_str, 'Time': time_str}, ignore_index=True)
        attendance_data.to_csv('attendance.csv', index=False)
        print(f'{name} marked as present at {time_str}')


I have Added this functionality into the video stream loop to log attendance when a face is recognized.

In [ ]:
# Within the face recognition loop, mark attendance
for (top, right, bottom, left), name in zip(face_locations, face_names):
    # Mark attendance only if face is recognized
    if name != "Unknown":
        mark_attendance(name)


## Step 5: Report Generation
Exported the attendance record as a CSV file.

In [ ]:
# Load the attendance data and display it
attendance_data = pd.read_csv('attendance_logs.csv')
print(attendance_data)


## Step 6: Optional - Building a Simple UI
I have built a simple Flask UI :

In [ ]:
from flask import Flask, render_template
import pandas as pd

app = Flask(__name__)

@app.route('/')
def home():
    # Load attendance data
    data = pd.read_csv('attendance_logs.csv')
    return render_template('attendance_logs.html', tables=[data.to_html()], titles=[''])

if __name__ == '__main__':
    app.run()
